# Lab For Experimentation

In [1]:
import json
import warnings
from typing import Any, Literal

import numpy as np
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)


# Demo (Prevents ruff from removing the unused module import)
name: Any
category: Literal["A", "B", "C"]
json.loads('{"name": "Bike Rental Prediction", "category": "A"}')

{'name': 'Bike Rental Prediction', 'category': 'A'}

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/Projects/Bike-Rental-Prediction


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [5]:
rng = np.random.default_rng(123)
x = rng.standard_normal(size=(1_000, 10))

X_train, X_test = train_test_split(x, test_size=0.2, random_state=123)
y_train = rng.standard_normal(size=(X_train.shape[0],))
y_test = rng.standard_normal(size=(X_test.shape[0],))

params: dict[str, Any] = {
    "n_estimators": 100,
    "max_depth": 10,
}

rf_reg = RandomForestRegressor(**params)

rf_reg.fit(X_train, y_train)
# rf_reg.score(X_test, y_test)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [6]:
rf_reg.feature_importances_

array([0.1132, 0.1001, 0.0748, 0.1156, 0.0906, 0.1151, 0.0931, 0.1039,
       0.0969, 0.0966])

In [7]:
import xgboost as xgb

In [8]:
from src.config import app_config
from src.exceptions import (
    CustomError,
    MLFlowConnectionError,
)
from src.ml.feature_engineering import FeatureEngineer
from src.ml.trainer import ModelTrainer

In [9]:
import httpx

port: int = 5001
url: str = f"http://localhost:{port}"


def check_mlflow(url: str, timeout: float = 2.0) -> bool:
    """
    Check MLflow endpoint accessibility and handle common httpx errors.

    Returns True if reachable (2xx), False otherwise.
    """
    try:
        resp = httpx.get(url, timeout=timeout)
        resp.raise_for_status()
        console.print("[success]MLflow is accessible")
        return True

    except httpx.HTTPStatusError as e:
        console.print(
            f"[error]MLflow returned non-2xx status: {e.response.status_code} — {e}"
        )
        return False

    except httpx.RequestError as e:
        # covers ConnectError, ReadTimeout, etc.
        console.print(f"[error]Network/connection error when contacting MLflow: {e}")
        return False

    except (MLFlowConnectionError, CustomError) as e:
        console.print(f"[error]Project-specific MLflow error: {e}")
        return False

    except Exception as e:
        console.print(f"[error]Unexpected error: {e}")
        return False


check_mlflow(url)

MLflow is accessible

True

In [10]:
fp: str = "../../../../Documents/data_dump/bike_data/database.parquet"
data: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data.shape}", style="info")

feat_eng = FeatureEngineer()
features_df: pl.DataFrame = feat_eng.create_all_features(
    data=data, config=app_config.feature_config
)
features_df.head()

Shape: (13903, 16)

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_lag_0hr,cnt_lag_1hr,cnt_lag_24hr,hr_lag_1hr,hr_lag_24hr,temp_lag_1hr,temp_lag_3hr,hum_lag_1hr,hum_lag_3hr,temp_rolling_mean_3hr,temp_rolling_median_3hr,temp_rolling_mean_6hr,temp_rolling_median_6hr,hum_rolling_mean_3hr,hum_rolling_median_3hr,hum_rolling_mean_6hr,hum_rolling_median_6hr,temp_plus_hum,hum_plus_hr,cnt_diff_1hr,cnt_diff_2hr,hr_diff_1hr,hr_diff_24hr,temp_diff_1hr,temp_diff_2hr,temp_diff_24hr,hum_diff_1hr,hum_diff_2hr,is_high_temp,is_high_hum,is_peak_hour,is_working_hour,is_business_hour,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i8,i8,i8,i8,i8,i64
1,1,0,0,6,0,1,0.24,0.81,1,0.0,1.0,-0.781831,0.62349,16,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.05,0.81,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,40
1,1,1,0,6,0,1,0.22,0.8,1,0.258819,0.965926,-0.781831,0.62349,40,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,1.8,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,32
1,1,2,0,6,0,1,0.22,0.8,1,0.5,0.866025,-0.781831,0.62349,32,40,16,1,0,0.22,0.24,0.8,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,2.8,-8,16,1,0,0.0,-0.02,0.22,0.0,-0.01,0,0,0,0,0,13
1,1,3,0,6,0,1,0.24,0.75,1,0.707107,0.707107,-0.781831,0.62349,13,32,16,2,0,0.22,0.24,0.8,0.81,0.226667,0.22,0.233333,0.24,0.783333,0.8,0.776667,0.775,0.99,3.75,-19,-27,1,0,0.02,0.02,0.22,-0.05,-0.05,0,0,0,0,0,1
1,1,4,0,6,0,1,0.24,0.75,1,0.866025,0.5,-0.781831,0.62349,1,13,16,3,0,0.24,0.22,0.75,0.8,0.233333,0.24,0.233333,0.24,0.766667,0.75,0.776667,0.775,0.99,4.75,-12,-31,1,0,0.0,0.02,0.22,0.0,-0.05,0,0,0,0,0,1


In [11]:
from src.ml.utils import split_into_train_val_test_sets
from src.schemas.types import ModelType

In [ ]:
params: dict[str, Any] = {
    "objective": "reg:squarederror",
    "learning_rate": 0.1,
    "max_depth": 6,
}
data_dict = split_into_train_val_test_sets(
    data=features_df, target_col="target", test_size=0.2
)
x_train, y_train = data_dict["x_train"], data_dict["y_train"]
x_val, y_val = data_dict["x_val"], data_dict["y_val"]
dtrain = xgb.DMatrix(x_train, y_train, enable_categorical=True)
dval = xgb.DMatrix(x_val, y_val, enable_categorical=True)

cv = xgb.cv(
    params=params,
    dtrain=dtrain,
    num_boost_round=500,
    nfold=5,
    metrics={"rmse"},
    early_stopping_rounds=10,
    seed=123,
)
# cv

In [12]:
trainer = ModelTrainer(data=features_df)
# trainer.hyperparameter_tuning_all_models()

2025-10-12 17:03:44 - mlflow_tracker - [INFO] - Set MLflow tracking URI to: http://localhost:5001
2025-10-12 17:03:45 - mlflow_tracker - [INFO] - Set experiment to: bike rental (ID: 1)
2025-10-12 17:03:45 - mlflow_tracker - [INFO] - Initialized MLFlowTracker with experiment: bike rental
Shapes -> x_train: (11260, 47), y_train: (11260,), x_val: (1252, 47), y_val: (1252,), x_test: (1391, 47), y_test: (1391,)
2025-10-12 17:03:45 - trainer - [INFO] - Data preparation complete.


In [23]:
result = trainer._hyperparameter_tuning_xgboost()
# result = trainer._hyperparameter_tuning_lightgbm()

result

2025-10-12 17:08:09 - trainer - [INFO] - 🚨 Starting hyperparameter tuning for XGBoost with 2 trials...
2025-10-12 17:08:10 - mlflow_tracker - [INFO] - Started MLflow run: a75fe729b8e64d36929607e818cb860b (name: run_2025-10-12T17:08:09)


[I 2025-10-12 17:08:10,196] A new study created in memory with name: no-name-406887b7-a939-42d5-aa6b-040581c3b4da


2025-10-12 17:08:10 - mlflow_tracker - [INFO] - Started MLflow run: 7f2e0cf43ae740e6a8dc4e3c5e3160db (name: run_2025-10-12T17:08:10)
[0]	validation-rmse:201.64793
[100]	validation-rmse:99.96270
[200]	validation-rmse:95.76086
[300]	validation-rmse:94.48302
[400]	validation-rmse:94.00693
[499]	validation-rmse:93.80237
Trial 0: Mean RMSE = 93.8
🏃 View run run_2025-10-12T17:08:10 at: http://localhost:5001/#/experiments/1/runs/7f2e0cf43ae740e6a8dc4e3c5e3160db
🧪 View experiment at: http://localhost:5001/#/experiments/1
2025-10-12 17:08:11 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED


[I 2025-10-12 17:08:11,689] Trial 0 finished with value: 93.8 and parameters: {'booster': 'gblinear', 'reg_lambda': 0.003907967156822882, 'reg_alpha': 8.63200816860254e-06, 'gamma': 9.614487780312039e-06, 'min_child_weight': 1, 'subsample': 0.9330880728874675, 'eta': 0.09183797741174658, 'n_estimators': 1077, 'grow_policy': 'lossguide', 'early_stopping_rounds': 44}. Best is trial 0 with value: 93.8.


2025-10-12 17:08:11 - trainer - [INFO] - Initial trial 0 achieved value: 93.8
2025-10-12 17:08:11 - mlflow_tracker - [INFO] - Started MLflow run: 4231689a1a0c4e11ba19971bbc4d734a (name: run_2025-10-12T17:08:11)
[0]	validation-rmse:238.64932
[100]	validation-rmse:107.32072
[200]	validation-rmse:80.33526
[300]	validation-rmse:70.93965
[400]	validation-rmse:67.27136
[499]	validation-rmse:65.43159
Trial 1: Mean RMSE = 65.43
🏃 View run run_2025-10-12T17:08:11 at: http://localhost:5001/#/experiments/1/runs/4231689a1a0c4e11ba19971bbc4d734a
🧪 View experiment at: http://localhost:5001/#/experiments/1
2025-10-12 17:08:13 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED


[I 2025-10-12 17:08:13,582] Trial 1 finished with value: 65.43 and parameters: {'booster': 'gbtree', 'reg_lambda': 6.690421166498802e-05, 'reg_alpha': 0.0014077923139972403, 'gamma': 0.0005268639929495403, 'min_child_weight': 3, 'subsample': 0.8059264473611898, 'eta': 0.01672929090610901, 'n_estimators': 473, 'grow_policy': 'lossguide', 'early_stopping_rounds': 42, 'max_depth': 4}. Best is trial 1 with value: 65.43.


2025-10-12 17:08:13 - trainer - [INFO] - Trial 1 achieved value: 65.43 with 30.2452% improvement
Best trial: 1 | Value: 65.43
[0]	validation-rmse:238.64932
[10]	validation-rmse:213.83706
[20]	validation-rmse:193.08523
[30]	validation-rmse:175.19494
[40]	validation-rmse:160.53519
[50]	validation-rmse:147.67195
[60]	validation-rmse:137.08946
[70]	validation-rmse:127.70117
[80]	validation-rmse:119.81424
[90]	validation-rmse:113.00221
[100]	validation-rmse:107.32072
[110]	validation-rmse:102.49885
[120]	validation-rmse:97.88963
[130]	validation-rmse:94.30404
[140]	validation-rmse:91.13270
[150]	validation-rmse:88.69829
[160]	validation-rmse:86.43537
[170]	validation-rmse:84.56235
[180]	validation-rmse:82.91783
[190]	validation-rmse:81.53278
[200]	validation-rmse:80.33526
[210]	validation-rmse:79.06666
[220]	validation-rmse:77.78195
[230]	validation-rmse:76.63408
[240]	validation-rmse:75.37712
[250]	validation-rmse:74.59233
[260]	validation-rmse:73.62021
[270]	validation-rmse:72.75607
[280]

{'run_id': 'a75fe729b8e64d36929607e818cb860b',
 'model_name': <ModelType.XGBOOST: 'XGBoostRegressor'>,
 'best_params': {'booster': 'gbtree',
  'reg_lambda': 6.690421166498802e-05,
  'reg_alpha': 0.0014077923139972403,
  'gamma': 0.0005268639929495403,
  'min_child_weight': 3,
  'subsample': 0.8059264473611898,
  'eta': 0.01672929090610901,
  'n_estimators': 473,
  'grow_policy': 'lossguide',
  'max_depth': 4,
  'seed': 42},
 'model_uri': 's3://mlflow-artifacts/1/a75fe729b8e64d36929607e818cb860b/artifacts/models/'}

In [ ]:
import mlflow

mlflow.end_run()

trainer.mlflow_tracker.load_model_artifact(
    run_id=result["run_id"],
    model_name=ModelType.XGBOOST.value,
    artifact_subpath="models",
)

In [ ]:
from src.exp_tracking.model_loader import (
    load_best_model,
    load_model_from_run,
)

In [ ]:
loaded_artifacts = load_model_from_run(
    run_id=result["run_id"], model_name=ModelType.XGBOOST
)

2025-10-12 17:35:26 - mlflow_tracker - [INFO] - Set MLflow tracking URI to: http://localhost:5001
2025-10-12 17:35:26 - mlflow_tracker - [INFO] - Set experiment to: bike rental (ID: 1)
2025-10-12 17:35:26 - mlflow_tracker - [INFO] - Initialized MLFlowTracker with experiment: bike rental


2025-10-12 17:35:27 - mlflow_tracker - [INFO] - Detected model file: 'ModelType.XGBOOST_2025-10-12T17:08:15_model.json' with extension 'json'
2025-10-12 17:35:27 - mlflow_tracker - [INFO] - ✅ Successfully loaded ModelType.XGBOOST model from run a75fe729b8e64d36929607e818cb860b


In [67]:
loaded = load_best_model(experiment_name=app_config.experiment_config.experiment_name)
# loaded

2025-10-12 17:40:07 - model_loader - [INFO] - Best run: 09b06e9f2aad455699941bbf34191e94 with best_rmse=56.88
2025-10-12 17:40:07 - mlflow_tracker - [INFO] - Set MLflow tracking URI to: http://localhost:5001
2025-10-12 17:40:07 - mlflow_tracker - [INFO] - Set experiment to: bike rental (ID: 1)
2025-10-12 17:40:07 - mlflow_tracker - [INFO] - Initialized MLFlowTracker with experiment: bike rental


2025-10-12 17:40:08 - mlflow_tracker - [INFO] - Detected model file: 'ModelType.LIGHTGBM_2025-10-12T17:04:15_model.txt' with extension 'txt'
2025-10-12 17:40:08 - mlflow_tracker - [INFO] - ✅ Successfully loaded ModelType.LIGHTGBM model from run 09b06e9f2aad455699941bbf34191e94
2025-10-12 17:40:08 - model_loader - [INFO] - ✅ Loaded best ModelType.LIGHTGBM model


In [68]:
console.print(loaded)

{
    'model': <lightgbm.basic.Booster object at 0x3072e4b60>,
    'metadata': {
        'model_type': 'ModelType.LIGHTGBM',
        'framework': 'lightgbm.basic',
        'class': 'Booster',
        'save_format': 'txt',
        'timestamp': '2025-10-12T17:04:15',
        'input_example': {
            'num_rows': 13903,
            'num_cols': 48,
            'dtypes': {
                'season': 'Int64',
                'mnth': 'Int64',
                'hr': 'Int64',
                'holiday': 'Int64',
                'weekday': 'Int64',
                'workingday': 'Int64',
                'weathersit': 'Int64',
                'temp': 'Float64',
                'hum': 'Float64',
                'is_weekend': 'Int8',
                'sin_hour': 'Float64',
                'cos_hour': 'Float64',
                'sin_weekday': 'Float64',
                'cos_weekday': 'Float64',
                'cnt_lag_0hr': 'Int64',
                'cnt_lag_1hr': 'Int64',
                'cnt_lag_24hr': 'Int64',
                'hr_lag_1hr': 'Int64',
                'hr_lag_24hr': 'Int64',
                'temp_lag_1hr': 'Float64',
                'temp_lag_3hr': 'Float64',
                'hum_lag_1hr': 'Float64',
                'hum_lag_3hr': 'Float64',
                'temp_rolling_mean_3hr': 'Float64',
                'temp_rolling_median_3hr': 'Float64',
                'temp_rolling_mean_6hr': 'Float64',
                'temp_rolling_median_6hr': 'Float64',
                'hum_rolling_mean_3hr': 'Float64',
                'hum_rolling_median_3hr': 'Float64',
                'hum_rolling_mean_6hr': 'Float64',
                'hum_rolling_median_6hr': 'Float64',
                'temp_plus_hum': 'Float64',
                'hum_plus_hr': 'Float64',
                'cnt_diff_1hr': 'Int64',
                'cnt_diff_2hr': 'Int64',
                'hr_diff_1hr': 'Int64',
                'hr_diff_24hr': 'Int64',
                'temp_diff_1hr': 'Float64',
                'temp_diff_2hr': 'Float64',
                'temp_diff_24hr': 'Float64',
                'hum_diff_1hr': 'Float64',
                'hum_diff_2hr': 'Float64',
                'is_high_temp': 'Int8',
                'is_high_hum': 'Int8',
                'is_peak_hour': 'Int8',
                'is_working_hour': 'Int8',
                'is_business_hour': 'Int8',
                'target': 'Int64'
            }
        }
    },
    'input_example': [
        {
            'season': 1,
            'mnth': 1,
            'hr': 0,
            'holiday': 0,
            'weekday': 6,
            'workingday': 0,
            'weathersit': 1,
            'temp': 0.24,
            'hum': 0.81,
            'is_weekend': 1,
            'sin_hour': 0.0,
            'cos_hour': 1.0,
            'sin_weekday': -0.7818314824680299,
            'cos_weekday': 0.6234898018587334,
            'cnt_lag_0hr': 16,
            'cnt_lag_1hr': 16,
            'cnt_lag_24hr': 16,
            'hr_lag_1hr': 0,
            'hr_lag_24hr': 0,
            'temp_lag_1hr': 0.24,
            'temp_lag_3hr': 0.24,
            'hum_lag_1hr': 0.81,
            'hum_lag_3hr': 0.81,
            'temp_rolling_mean_3hr': 0.22666666666666666,
            'temp_rolling_median_3hr': 0.22,
            'temp_rolling_mean_6hr': 0.2333333333333333,
            'temp_rolling_median_6hr': 0.24,
            'hum_rolling_mean_3hr': 0.8033333333333333,
            'hum_rolling_median_3hr': 0.8,
            'hum_rolling_mean_6hr': 0.7766666666666667,
            'hum_rolling_median_6hr': 0.775,
            'temp_plus_hum': 1.05,
            'hum_plus_hr': 0.81,
            'cnt_diff_1hr': 24,
            'cnt_diff_2hr': 16,
            'hr_diff_1hr': 1,
            'hr_diff_24hr': 0,
            'temp_diff_1hr': -0.01999999999999999,
            'temp_diff_2hr': -0.01999999999999999,
            'temp_diff_24hr': 0.22000000000000003,
            'hum_diff_1hr': -0.010000000000000009,
            'hum_diff_2hr': -0.010000000000